In [1]:

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
from datasets import DatasetDict
import datasets

In [2]:
dataset_train = load_dataset('csv', data_files='train_all_untailored_seq.csv')
dataset_train=dataset_train['train'].rename_column("Sequence", "Seq")


Using custom data configuration default-a919968db2e3dae8


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/lu/.cache/huggingface/datasets/csv/default-a919968db2e3dae8/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
#Train test split

dataset=dataset_train.train_test_split(test_size=0.01, shuffle=True)

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Seq', 'Exp'],
        num_rows: 6671865
    })
    test: Dataset({
        features: ['Seq', 'Exp'],
        num_rows: 67393
    })
})

In [9]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding,AutoConfig
from torch.utils.data import DataLoader


BASE_MODEL = "bert-base-uncased"
LEARNING_RATE = 1e-5
MAX_LENGTH = 128
BATCH_SIZE = 128
EPOCHS = 3


model_checkpoint = "bert-base-uncased"

config = AutoConfig.from_pretrained(model_checkpoint,num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_config(config)

In [10]:
from re import findall
import itertools  
#Defining the function that splits the sequences into nuclitides seperated by spaces

def textProcess(s,k,fill=' '):
  return ' '.join(map(''.join, itertools.zip_longest(*[iter(s)]*k, fillvalue=fill)))

In [ ]:
#Preprocess the input and tokenize the sequences, note that only the sequences and their expression values are used

def preprocess_function(examples):
    text=examples['Seq']
    label=examples['Exp']
    text=textProcess(text,1)
    examples["text"]=text
    examples = tokenizer(text, truncation=True, padding="max_length", max_length=MAX_LENGTH)
    # Change this to real number
    examples["label"] = float(label)
    return examples

tokenized_dataset=DatasetDict()
for split in dataset:
    tokenized_dataset[split] = dataset[split].map(preprocess_function,num_proc=48)

Parameter 'function'=<function preprocess_function at 0x7fe0d370f370> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#21:   0%|          | 0/138997 [00:00<?, ?ex/s]

#27:   0%|          | 0/138997 [00:00<?, ?ex/s]

#28:   0%|          | 0/138997 [00:00<?, ?ex/s]

#33:   0%|          | 0/138997 [00:00<?, ?ex/s]

...

#21:   0%|          | 0/1404 [00:00<?, ?ex/s]

#46:   0%|          | 0/1404 [00:00<?, ?ex/s]

#45:   0%|          | 0/1404 [00:00<?, ?ex/s]

#47:   0%|          | 0/1404 [00:00<?, ?ex/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
tokenized_dataset

In [ ]:
def selectColumns(dataset,columns=["input_ids","token_type_ids","attention_mask","label"]):
    cols_to_remove= dataset.column_names
    for c in columns:
        cols_to_remove.remove(c)
    return dataset.remove_columns(cols_to_remove)


In [ ]:
for split in tokenized_dataset:
    tokenized_dataset[split]=selectColumns(tokenized_dataset[split])

save

In [ ]:
tokenized_dataset

In [ ]:
!nvidia-smi -L


In [ ]:

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):save
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)
    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    return {"mse": mse, "mae": mae, "r2": r2}


In [ ]:
from transformers import TrainingArguments

EPOCHS = 3
BATCH_SIZE=36
training_args = TrainingArguments(
    f"zluvolyote/DEREXP",
    save_total_limit=3,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    optim="adamw_torch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="r2",
    load_best_model_at_end=True,
    weight_decay=0.01,
    dataloader_num_workers= 2,
    remove_unused_columns=True,

)

Make sure your cuda version supports 'sm_86'

In [ ]:
import torch
print(torch.cuda.get_arch_list())

In [ ]:
import torch  

from transformers import Trainer
class RegressionTrainer(Trainer):save
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        save
        loss = torch.nn.functional.mse_loss(logits, labels.float())
        return (loss, outputs) if return_outputs else loss

In [ ]:
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics_for_regression,
    tokenizer=tokenizer,
)



In [ ]:
!nvidia-smi

In [ ]:
print(torch.cuda.current_device())
print(torch.cuda.device_count())

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('./bert_epoch3_r52')